In [24]:
# Library needed to use JuMP with GLPK solver
using JuMP
using GLPK

const OPTIMAL = JuMP.MathOptInterface.OPTIMAL
const INFEASIBLE = JuMP.MathOptInterface.INFEASIBLE
const UNBOUNDED = JuMP.MathOptInterface.DUAL_INFEASIBLE;

#V is the list of all vertexs with their coordinates x, y
#S is the list that containt the type of each vertex : 
#    0 : optionnal stations
#    1 : needed stations
#    2 : starting station
#    3 : arriving station
#G is the adjacancy matrix, used to know all edges

#the main fonction, that returns a sort of adjacancy matrix (oriented)

function resolve(V, S, G)
    
    #height and width of G
    x = length(G[:,1])
    y = length(G[1,:])
    
    #Calculation of distances between vertexs
    Distance = zeros(Float64, x, y)
    for i in 1:x
        for j in 1:y
            if (G[i, j] == 0)
                Distance[i, j] = 0
            else    
                Distance[i, j] = hypot(V[i, 1] - V[j, 1], V[i, 2] - V[j, 2])
            end
        end
    end
    
    #Vstart is the index of the starting station, Vend the index of the ending station. Nbr is the number of needed stations
    nbr = 0
    Vstart = 1
    Vend = 1
    for i in 1:x
        if (S[i] == 1)
            nbr = nbr + 1
        end
        if (S[i] == 2)
            Vstart = i
        end
        if (S[i] == 3)
            Vend = i
        end
    end
    
    # Creation of the model
    m = Model(with_optimizer(GLPK.Optimizer))

    # Creatuion of the result matrix
    @variable(m, R[1:y,1:x] >= 0, Int)
    # Creation of a matrix that will help us to get rid of the sub-loops
    @variable(m, Z[1:y,1:x] >= 0, Int)
   
    # Objective function
    @objective(m, Min, sum(R .* Distance))
   
    
    #the first contraint allows us to only use edges that exist
    #the second one makes sure that we can't use the same edge in both direction
    for i in 1:x
        for j in 1:y
            if (G[j, i] == 0)
                @constraint(m, R[j, i] == 0)    
            elseif (i > j)
                @constraint(m, R[j, i] + R[i, j] <= 1)
            end
        end
    end

    
    for i in 1:x
        #starting vertex : we need an edge that come out of the vertex
        if (S[i] == 2)
            @constraint(m, sum(R[i, :]) == 1)
            @constraint(m, sum(R[:, i]) == 0)
        #ending vertex : we need an edge that come in the vertex
        elseif (S[i] == 3)
            @constraint(m, sum(R[:, i]) == 1)
            @constraint(m, sum(R[i, :]) == 0)
        #needed vertex : we need an edge that come in and an other that come out of the vertex
        elseif (S[i] == 1)
            @constraint(m, sum(R[:, i]) == 1)
            @constraint(m, sum(R[i, :]) == 1)
        #steiner vertex : we need an edge that come in AND an edge that come out, if we use it, or nothing
        elseif (S[i] == 0)
            @constraint(m, sum(R[i, :] - R[:, i]) == 0)
            @constraint(m, sum(R[i, :]) <= 1)
        end
    end

    #constraints that help against sub-loops
 
    #to get ride of the sub-loops, we use a flow that starts at the starting vertex, which is equal to the number
    #of needed vertex + 1, and we reduce it each times it goes through a needed vertex
    #that makes sure that the path will go through each needed vertex
    
    for i in 1:x
        #the flow that goes out of the starting vertex has to be nbr + 1
        if (S[i] == 2)
            @constraint(m, sum(Z[i, :]) == nbr + 1)
        end
        #the flow that goes in the ending vertex has to be 1
        if (S[i] == 3)
            @constraint(m, sum(Z[:, i]) == 1)
        end
        #the flow that goes out of a needed vertex has to be the flow that arrive at him minus one 
        if (S[i] == 1)
            @constraint(m, sum(Z[i, :]) + 1 == sum(Z[:, i]))
        end
        #the flow that goes out of a needed vertex has to be same as the flow that arrive at him
        if (S[i] == 0)
            @constraint(m, sum(Z[i, :]) == sum(Z[:, i]))
        end
    end
    
    for i in 1:x
        for j in 1:y
            if (i != j && j != Vstart)
                #we only have flows only in used vertex
                @constraint(m, Z[i, j] <= (nbr+1) * (R[i, j]))
            end
        end
    end

    #Resolving
    optimize!(m)

    status = termination_status(m)

    if status == INFEASIBLE
        println("The instance is not resolvable")
    elseif status == UNBOUNDED
        println("How did you manage to get negative distances?")
    elseif status == OPTIMAL
        println("Optimum = ", JuMP.objective_value(m))
        println("Solution :")
        for i in 1:x
            for j in 1:y
               print(JuMP.value(R[i, j]), " ")
            end
            println()
        end
    else
        println("Error during resolving")
    end
end

resolve (generic function with 1 method)

In [25]:
V = [0.0 0.0; 0.0 4.0; 2.0 2.0; 4.0 0.0; 4.0 4.0]
S = [2 1 0 0 3]
G = [0 1 1 1 0;
     1 0 1 0 1;
     1 1 0 1 1;
     1 0 1 0 1;
     0 1 1 1 0]
resolve(V, S, G)

Optimum = 8.0
Solution :
0.0 1.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 1.0 
0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 


In [26]:
V = [0.0 0.0; 0.0 4.0; 2.0 2.0; 4.0 0.0; 4.0 4.0]
S = [2 0 0 0 3]
G = [0 1 1 1 0;
     1 0 1 0 1;
     1 1 0 1 1;
     1 0 1 0 1;
     0 1 1 1 0]
resolve(V, S, G)

Optimum = 5.656854249492381
Solution :
0.0 0.0 1.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 1.0 
0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 


In [27]:
V = [0.0 0.0; 1.0 1.0; 3.0 3.0; 5.0 5.0; 6.0 6.0]
S = [2 0 0 0 3]
G = [0 1 0 0 0;
     1 0 1 0 0;
     0 1 0 1 0;
     0 0 1 0 1;
     0 0 0 1 0]
resolve(V, S, G)

Optimum = 8.485281374238571
Solution :
0.0 1.0 0.0 0.0 0.0 
0.0 0.0 1.0 0.0 0.0 
0.0 0.0 0.0 1.0 0.0 
0.0 0.0 0.0 0.0 1.0 
0.0 0.0 0.0 0.0 0.0 


In [28]:
V = [202.0 0.0; 200.0 0.0; 2.0 0.0; 0.0 0.0; 1.0 1.0; 201.0 1.0]
S = [0 1 0 3 2 1]
G = [0 1 0 0 0 1;
     1 0 1 0 0 1;
     0 1 0 1 1 0;
     0 0 1 0 1 0;
     0 0 1 1 0 1;
     1 1 0 0 1 0]
resolve(V, S, G)

Optimum = 401.4142135623731
Solution :
0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 1.0 0.0 0.0 0.0 
0.0 0.0 0.0 1.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 1.0 
0.0 1.0 0.0 0.0 0.0 0.0 


In [29]:
V = [0.0 0.0; 1.0 0.0; 1.0 -1.0; 2.0 0.0; 1.0 -15.0]
S = [2 1 1 3 0]
G = [0 1 0 0 1;
     1 0 1 1 1
     0 1 0 0 1
     0 1 0 0 1
     1 1 1 1 0]
resolve(V, S, G)

Optimum = 31.03329637837291
Solution :
0.0 1.0 0.0 0.0 0.0 
0.0 0.0 1.0 0.0 0.0 
0.0 0.0 0.0 0.0 1.0 
0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 1.0 0.0 
